# Uber Case Study Data Transformation

### Uber Pickup Data

In [1]:
import numpy as np 
import pandas as pd

In [2]:
# load Uber data
# Weather data only includes 4 months

path=r'C:\Users\owner\Desktop\Uber Case Study\Uber Data'

files = ['uber-raw-data-apr14.csv',
        'uber-raw-data-may14.csv',
        'uber-raw-data-jun14.csv',
        'uber-raw-data-jul14.csv']

final = pd.DataFrame()

for file in files:
    df = pd.read_csv(path+"/"+file, encoding = 'utf-8')
    final = pd.concat([final,df])

uber = final.copy()

In [3]:
# Convert to date/time
uber['Date/Time']=pd.to_datetime(uber['Date/Time'],format='%m/%d/%Y %H:%M:%S')

# Seperate date elements into different columns
uber['Date']=uber['Date/Time'].dt.date.astype('datetime64')
uber['Time']=uber['Date/Time'].dt.time
uber['day_name'] = uber['Date/Time'].dt.day_name()
uber['day'] = uber['Date/Time'].dt.day
uber['month'] = uber['Date/Time'].dt.month
uber['hour'] = uber['Date/Time'].dt.hour

# Create a unique identifier for each pickup
uber['Pickup_id'] = np.arange(len(uber))+1000

In [4]:
uber.head()

,Date/Time,Lat,Lon,Base,Date,Time,day_name,day,month,hour,Pickup_id
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,2014-04-01,00:11:00,Tuesday,1,4,0,1000
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,2014-04-01,00:17:00,Tuesday,1,4,0,1001
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,2014-04-01,00:21:00,Tuesday,1,4,0,1002
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,2014-04-01,00:28:00,Tuesday,1,4,0,1003
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,2014-04-01,00:33:00,Tuesday,1,4,0,1004


### Weather Data

In [5]:
# Load weather data
weather = pd.read_excel(r'C:\Users\owner\Desktop\Uber Case Study\Weather Data\nyweatherclean.xlsx')

C:\Users\owner\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [6]:
# Drop unneeded columns
weather.drop(columns = ['Day', 'Month', 'Year', ], inplace=True)

# Replace trace (T) with zero
weather['Precip.(mm)_sum'] = weather['Precip.(mm)_sum'].replace('T', 0)

# Convert Events into seperate encoded columns
weather['Events'] = weather['Events'].fillna("None")
weather['Event_Rain'] = np.where(weather['Events'].str.contains("Rain"), 1, 0)
weather['Event_Fog'] = pd.np.where(weather['Events'].str.contains("Fog"), 1, 0)
weather['Event_Snow'] = pd.np.where(weather['Events'].str.contains("Snow"), 1, 0)
weather['Event_Thunderstorm'] = pd.np.where(weather['Events'].str.contains("Thunderstorm"), 1, 0)

C:\Users\owner\AppData\Local\Temp\ipykernel_18028\2091772151.py:10: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  weather['Event_Fog'] = pd.np.where(weather['Events'].str.contains("Fog"), 1, 0)
C:\Users\owner\AppData\Local\Temp\ipykernel_18028\2091772151.py:11: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  weather['Event_Snow'] = pd.np.where(weather['Events'].str.contains("Snow"), 1, 0)
C:\Users\owner\AppData\Local\Temp\ipykernel_18028\2091772151.py:12: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  weather['Event_Thunderstorm'] = pd.np.where(weather['Events'].str.contains("Thunderstorm"), 1, 0)


### Join Uber Pickup and Weather Data into one dataset

In [7]:
uber_weather = uber.merge(weather, on = 'Date', how = 'left')

In [10]:
uber_weather.shape

(2676916, 35)

In [17]:
uber_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2676916 entries, 0 to 2676915
Data columns (total 35 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Date/Time                 datetime64[ns]
 1   Lat                       float64       
 2   Lon                       float64       
 3   Base                      object        
 4   Date                      datetime64[ns]
 5   Time                      object        
 6   day_name                  object        
 7   day                       int64         
 8   month                     int64         
 9   hour                      int64         
 10  Pickup_id                 int32         
 11  Temp.(°C)_high            int64         
 12  Temp.(°C)_avg             int64         
 13  Temp.(°C)_low             int64         
 14  Dew Point(°C)_high        int64         
 15  Dew Point(°C)_avg         int64         
 16  Dew Point(°C)_low         int64         
 17  Humidity

In [9]:
uber_weather.head()

,Date/Time,Lat,Lon,Base,Date,Time,day_name,day,month,hour,...,Visibility(km)_low,Wind(km/h)_avg,Wind(km/h)_Low,Wind(km/h)_high,Precip.(mm)_sum,Events,Event_Rain,Event_Fog,Event_Snow,Event_Thunderstorm
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,2014-04-01,00:11:00,Tuesday,1,4,0,...,16,26,16,40,0.0,None,0,0,0,0
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,2014-04-01,00:17:00,Tuesday,1,4,0,...,16,26,16,40,0.0,None,0,0,0,0
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,2014-04-01,00:21:00,Tuesday,1,4,0,...,16,26,16,40,0.0,None,0,0,0,0
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,2014-04-01,00:28:00,Tuesday,1,4,0,...,16,26,16,40,0.0,None,0,0,0,0
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,2014-04-01,00:33:00,Tuesday,1,4,0,...,16,26,16,40,0.0,None,0,0,0,0


In [11]:
# Final Combined Dataset
uber_weather.to_csv(r'C:\Users\owner\Desktop\Uber Case Study\Uber Data\Uber_Weather.csv', index=False)